In [4]:
import paramiko
import os.path as path
import os

from getpass import getpass
from pathlib import Path


theFile = input("What is the name of the results file")



if path.exists(theFile) == True:
    fileExist = input("This file already exists would you like to delete it")
    if fileExist == "y":
        os.remove(theFile)
else:
    print("The file does not already exist")

#Create the password prompt
thePass = getpass(prompt="Please enter your SSH password: ")

#Host Infomation
host = "192.168.6.71"
port = "2222"
username = "sebastian.meredith"
password = thePass

try:

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authenication Failed.")

In [5]:
sftp = ssh.open_sftp()
# File to traverse a given directory andf it's subdirs and retrieve all the files
command = ['sudo -S ps -ef' , 'sudo -S /home/ubuntu/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin' , 'sudo -S lsof -p 148658', 'sudo -S lsof -i :8080','sudo -S grep p.dalton /etc/passwd','sudo -S last p.dalton','grep http-alt /etc/services', 'python3 download.py -d /home/p.dalton', 'sudo -S dir /home/p.dalton']
files = ['/usr/lib/x86_64-linux-gnu/libc-2.31.so','/home/p.dalton/.d.log','/usr/local/bin/ls','/usr/lib/x86_64-linux-gnu/libpthread-2.31.so','/usr/lib/x86_64-linux-gnu/ld-2.31.so','/home/p.dalton/test-lk']
for eachCMD in command:
    # Get the output from the command
    (stdin, stdout, stderr) = ssh.exec_command(eachCMD)
    

    stdin.write(thePass + "\n")
    # Get result from stdout
    lines = stdout.readlines()
    #print(lines)

    #Convert the list to a string 
    output = ''.join(lines)
    # Header output
    sepHeader = '' + '### BEGIN ' + eachCMD + '###\n\n'
    # Footer
    sepFooter = '' + '### END ' + eachCMD + '###\n\n'

    cmd_output = sepHeader + output + sepFooter

    # Save the cmd output to a file
    with open ('ssh_results.txt', 'a') as f:
        f.write(cmd_output)
    #print(output)

for eachFile in files:
    x = eachFile.split ("/")
    filename = x [-1]
    
    local_path = "remote_files/" + filename
    sftp.get(eachFile, local_path)
ssh.close()



OSError: Failure